In [135]:
import pandas
import numpy as np
import warnings

In [136]:
warnings.filterwarnings("ignore")
df = pandas.read_csv('wbcomps.csv',sep=";")
df

,Thumb,SKU,Name,Revenue,Sales,Category,Category Position,Brand,Seller,Balance,Comments,Rating,Final price,SPP Price,Days in stock,Turnover,Turnover days
0,https://basket-01.wb.ru/vol30/part3048/3048451...,3048451,Кольцо женское золотое 585 пробы с камнем,4122674,497,Ювелирные изделия/Украшения из золота/Ювелирно...,446,SOKOLOV,АО Лакса Трейдинг,281,2354,5,11949,11949,30,"1,26",24
1,https://basket-08.wb.ru/vol1132/part113256/113...,113256091,Кольцо золотое 585 пробы с фианитом помолвочное,3827480,667,Ювелирные изделия/Украшения из золота/Ювелирно...,14,SOKOLOV,АО Лакса Трейдинг,408,3645,5,5801,5801,30,"1,5",20
2,https://basket-12.wb.ru/vol1804/part180497/180...,180497628,Кольцо Возвышенное красное сердце Подарок,3726944,2260,Ювелирные изделия/Кольца,4,PANDORA,ИП Зубелевич Т Г,600,1240,5,1515,1515,30,"1,4",21
3,https://basket-01.wb.ru/vol34/part3436/3436272...,3436272,Ювелирная цепочка на шею серебро 925,3247398,5481,Ювелирные изделия/Украшения из серебра/Ювелирн...,143,SOKOLOV,АО Лакса Трейдинг,3924,14806,5,844,844,30,"1,57",19
4,https://basket-01.wb.ru/vol29/part2900/2900455...,2900455,Цепочка на шею из серебра 925,3029294,4561,Ювелирные изделия/Украшения из серебра/Ювелирн...,200,SOKOLOV,АО Лакса Трейдинг,2377,5083,5,943,943,30,"2,25",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130779,https://basket-10.wb.ru/vol1561/part156145/156...,156145094,Ювелирный чокер с натуральным камнем,0,0,Ювелирные изделия/Украшения из серебра/Ювелирн...,226,L'ATTRICE,ООО Латтриче,1,0,0,2199,2199,30,0,0
130780,https://basket-03.wb.ru/vol397/part39713/39713...,39713384,Часы наручные мужские из серебра 925,0,0,Ювелирные изделия/Часы/Ювелирные часы,14,SOKOLOV,АО Лакса Трейдинг,4,177,5,11697,11697,3,0,0
130781,https://basket-11.wb.ru/vol1628/part162899/162...,162899864,Колье золотое с эмалью и аметистом,0,0,NaN,-1,Русские самоцветы,ООО ИЮД Русские Самоцветы 1912,0,0,0,135941,135941,0,0,0
130782,https://basket-05.wb.ru/vol765/part76578/76578...,76578794,Ювелирная цепочка,0,0,Ювелирные изделия/Украшения из серебра/Ювелирн...,4322,Кимберли,Достовалова Ирина Геннадьевна,20,0,0,1202,1202,30,0,0


In [137]:
def tabl(dataf,stroka,probas):
    dataf['Category'] = dataf['Category'].fillna('Нет')
    dataf = dataf.query(f'Category.str.contains("{stroka}|Нет")')
    dataf = dataf[dataf['Name'].str.contains(f"{stroka}|{probas}")|~dataf['Category'].str.contains("Нет")].reset_index(drop=True)
    for i in range(len(dataf)):
        if dataf['Category'][i] != 'Нет':
            dataf.at[i,'Category'] = dataf['Category'][i][dataf['Category'][i].rfind("/")+1:]
        elif "кольцо" in dataf['Name'][i].lower():
            dataf.at[i,'Category'] = "Ювелирное кольцо"
        elif "браслет" in dataf['Name'][i].lower():
            dataf.at[i,'Category'] = "Ювелирный браслет"
        elif "цеп" in dataf['Name'][i].lower():
            dataf.at[i,'Category'] = "Ювелирная цепочка"
        elif "серьг" in dataf['Name'][i].lower():
            dataf.at[i,'Category'] = "Ювелирные серьги"
        elif "подвес" in dataf['Name'][i].lower():
            dataf.at[i,'Category'] = "Ювелирная подвеска"
    dataf['Category'] = np.where(dataf['Category'].str.contains('кольц|браслет|цеп|серьг|подвес'),dataf['Category'],"Другое")
    dataf['Catcount'] = np.where(dataf['Sales']>0,1,0)
    dataf['stockcard'] = np.where(dataf['Balance']>0,1,0)
    ser2 = dataf.groupby('Category').agg({'Category':'count','Balance':'sum','stockcard':'sum','Catcount':'sum','Sales':'sum','Revenue':'sum'})
    prodsdf = pandas.DataFrame(columns=['prod1','prod2','prod3','prod1v','prod2v','prod3v'])
    for i in dataf['Category'].unique():
        prods = list()
        prods2 = list()
        s = dataf[dataf['Category'] == i]
        s1 = s.groupby('Seller').agg({'Sales':'sum','Seller':'max'}).sort_values(by='Sales', ascending=False)
        s2 = s.groupby('Seller').agg({'Revenue':'sum','Seller':'max'}).sort_values(by='Revenue', ascending=False)
        for i in range(3):
            prods.append(s1.iloc[i]['Seller'])
            prods2.append(s2.iloc[i]['Seller'])
        prods = prods + prods2
        prodsdf.loc[len(prodsdf)] = prods
    for i in prodsdf.columns.values:
        ser2[i] = prodsdf[i].values
    return ser2

In [138]:
word = "серебр"
proba = 925
ser = tabl(df,word,proba)
ser.to_excel(f"wb_{word}.xlsx")

In [139]:
df1 = df[~df['Category'].str.contains(word)]
word = "золот"
proba = 585
ser = tabl(df1,word,proba)
ser.to_excel(f"wb_{word}.xlsx")

In [140]:
df['Catcount'] = np.where(df['Sales']>0,1,0)
df['stockcard'] = np.where(df['Balance']>0,1,0)
comps = df.groupby('Seller').agg({'Name':'count','Balance':'sum','stockcard':'sum','Catcount':'sum','Revenue':'sum','Sales':'sum'}).sort_values(by='Sales', ascending=False).reset_index()
comps.columns = ['Продавец','Количество карточек','Товара в наличии','Карточек с остатком','Карточек с продажами','Выручка','Количество продаж']
comps = comps.head(20)
comps.to_excel("wb_comps.xlsx")

In [141]:
sells = df[['Name','Seller','Balance','Sales','Revenue','Final price']]
sells = sells.head(20)
sells.to_excel("wb_sells.xlsx")